## 🔧 Data Cleaning

The original dataset from Our World in Data (OWID) includes over 100 columns and global + country-level data from 1965 onward. To prepare for clean energy displacement analysis, the following data cleaning steps were performed:

### 1. Filter for Global Data
- Extract only rows where `country == "World"` to focus on global electricity trends.

### 2. Select Relevant Columns
To analyze energy transition pathways, we retain the following electricity generation columns (in terawatt-hours, TWh):

#### 🔥 Fossil Fuel Sources
- `coal_electricity`
- `gas_electricity`
- `oil_electricity`

#### ✅ Renewable Sources
- `solar_electricity`
- `wind_electricity`
- `hydro_electricity`
- `biofuel_electricity`
- `other_renewable_electricity` (e.g., geothermal, tidal)

#### ⚛️ Low-Carbon (Non-Renewable)
- `nuclear_electricity`

#### ⚡ Total Generation
- `electricity_generation` (used for share calculations)

### 3. Create Derived Columns
Add the following calculated fields:
- `total_renewable_electricity = solar + wind + hydro + biofuel + other_renewable`
- `total_fossil_electricity = coal + gas + oil`
- `total_low_carbon_electricity = total_renewable_electricity + nuclear_electricity`
- `clean_share = total_low_carbon_electricity / electricity_generation`
- `fossil_share = total_fossil_electricity / electricity_generation`

### 4. Handle Missing or Incomplete Data
- Fill missing values with `0` where appropriate.
- Drop rows for years where key electricity generation fields are entirely missing.

### 5. Create dataset highlighting IEA targets
- Go through cleaning process of IEA target dataset, and create new dataset of current, and target values.

### 5. Export for Tableau
- Save the cleaned dataset as `energy-data-cleaned.csv` and `future-energy-targets.csv`
- Format: one row per year, one column per electricity source and derived metric.

> Cleaning was performed using Python (`pandas`).

In [78]:
# 📦 Import standard libraries
import pandas as pd
import numpy as np

# 📊 For quick visual checks
import matplotlib.pyplot as plt
import seaborn as sns

# 🔧 Display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [79]:
# Read in .csv file and store into dataframe
df = pd.read_csv("../data/owid-energy-data.csv")


In [80]:
# Extract only rows where country == "World" to focus on global electricity trends.
df = df[df['country'] == 'World']


In [81]:
# Select relevant columns

columns = [
    'year',
    'coal_electricity', 'gas_electricity', 'oil_electricity',
    'solar_electricity', 'wind_electricity', 'hydro_electricity',
    'biofuel_electricity', 'other_renewable_electricity',
    'nuclear_electricity',
    'electricity_generation'
]
df = df[columns]

# fill NaN values with 0

df = df.fillna(0)

# drop rows where the electricity_generation = 0

df = df[df['electricity_generation'] > 0]

In [82]:
# Create derived columns

# total amount of energy through renewable sources
df['total_renewable_electricity'] = (
    df['solar_electricity'] + df['wind_electricity'] +
    df['hydro_electricity'] + df['biofuel_electricity'] +
    df['other_renewable_electricity'])

# total amount of energy through fossil-fuel sources
df['total_fossil_electricity'] = (df['coal_electricity'] + df['gas_electricity'] + df['oil_electricity'])

# total low carbon energy
df['total_low_carbon_electricity'] = (df['total_renewable_electricity'] + df['nuclear_electricity'])

# fossil share of total energy
df['fossil_share_of_generation'] = (df['total_fossil_electricity'] / df['electricity_generation'])

# renewable share of total energy
df['renewable_share_of_generation'] = (df['total_renewable_electricity'] / df['electricity_generation'])

# low carbon share of total energy
df['low_carbon_share_of_generation'] = (df['total_low_carbon_electricity'] / df['electricity_generation'])

In [84]:
# reset the index
df = df.reset_index(drop = True)

### 🔧 Work on second dataset

In [143]:
# Read in new dataset
df_target = pd.read_csv("../data/NZE2021_AnnexA.csv")

In [144]:
# Filter on electricity generation
df_target = df_target[df_target['Flow'] == "Electricity generation"]

# Filter where year > 2023
df_target = df_target[df_target['Year'] > 2025]

# Filter only solar/wind/hydro/fossil fuel
energy = ['Solar PV', 'Wind', 'Hydro', 'Unabated fossil fuels']
df_target = df_target[df_target['Product'].isin(energy)]

# remove any unnecessary columns
df_target = df_target[['Product', 'Year', 'Value']]


In [147]:
# Transform dataframe using pivot
df_target = df_target.pivot(index="Year", columns="Product", values="Value").reset_index()

In [150]:
# Remove "product" from index column
df_target.columns.name = None  # <<< This line removes the 'Product' label from the column

In [153]:
# set index to the year
df_target = df_target.set_index('Year')

In [155]:
df_target

,Hydro,Solar PV,Unabated fossil fuels,Wind
Year,,,,
2030,5870.0,6970.0,9358.0,8008.0
2040,7445.0,17031.0,632.0,18787.0
2050,8461.0,23468.0,259.0,24785.0


In [156]:
# sort columns
df_target = df_target[["Wind", "Solar PV", "Hydro", "Unabated fossil fuels"]]

In [158]:
# rename columns

df_target = df_target.rename(columns={"Solar PV": "Solar",
                                      "Unabated fossil fuels": "Fossil Fuels"})

In [163]:
# save the dataframe

df.to_csv("../data/energy-data-cleaned.csv", index=True)
df_target.to_csv("../data/future-energy-targets.csv", index=True)

In [164]:
df

,year,coal_electricity,gas_electricity,oil_electricity,solar_electricity,wind_electricity,hydro_electricity,biofuel_electricity,other_renewable_electricity,nuclear_electricity,electricity_generation,total_renewable_electricity,total_fossil_electricity,total_low_carbon_electricity,fossil_share_of_generation,renewable_share_of_generation,low_carbon_share_of_generation
0,1985,3748.385,1426.332,1110.757,0.012,0.064,1979.244,0.00,78.703,1488.922,9886.064,2058.023,6285.474,3546.945,0.635791,0.208174,0.358782
1,1986,3839.010,1432.692,1168.273,0.015,0.139,2006.073,0.00,85.443,1594.736,10180.790,2091.670,6439.975,3686.406,0.632561,0.205453,0.362094
2,1987,4058.077,1516.510,1183.175,0.011,0.195,2033.192,0.00,91.516,1734.733,10670.628,2124.914,6757.762,3859.647,0.633305,0.199137,0.361708
3,1988,4200.674,1540.980,1256.525,0.010,0.332,2098.349,0.00,94.161,1891.249,11140.627,2192.852,6998.179,4084.101,0.628167,0.196834,0.366595
4,1989,4376.982,1728.554,1350.200,0.262,2.650,2087.587,0.00,104.672,1945.011,11657.959,2195.171,7455.736,4140.182,0.639540,0.188298,0.355138
5,1990,4460.242,1788.729,1366.023,0.388,3.632,2158.619,0.00,117.224,2000.596,11961.343,2279.863,7614.994,4280.459,0.636634,0.190603,0.357858
6,1991,4557.066,1814.147,1351.379,0.505,4.086,2208.542,0.00,122.263,2096.310,12222.450,2335.396,7722.592,4431.706,0.631837,0.191074,0.362587
7,1992,4649.810,1828.429,1329.644,0.467,4.733,2208.314,0.00,131.160,2112.223,12336.336,2344.674,7807.883,4456.897,0.632918,0.190062,0.361282
8,1993,4727.774,1862.431,1268.480,0.557,5.704,2341.319,0.00,135.481,2184.965,12599.859,2483.061,7858.685,4668.026,0.623712,0.197071,0.370482
9,1994,4891.716,1923.732,1303.714,0.597,7.132,2355.905,0.00,140.677,2225.979,12923.506,2504.311,8119.162,4730.290,0.628248,0.193780,0.366022
